In [31]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [32]:
import boto3
from datetime import datetime
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements
from sagemaker.predictor import Predictor
from sagemaker.enums import EndpointType
from sagemaker.model import Model
from sagemaker.session import Session

In [33]:
# Create a variable w/ the model S3 URL
sagemaker_client = boto3.client("sagemaker")

# The name of your S3 bucket:
s3_bucket = "x-beagle"
# The directory within your S3 bucket your model is stored in:
bucket_prefix = "models/"
# The file name of your model artifact:
model_filename = "model.tar.gz"
# Relative S3 path:
model_s3_key = f"{bucket_prefix}/"+model_filename
# Combine bucket name, model file name, and relate S3 path to create S3 model URL:
model_url = f"s3://{s3_bucket}/{model_s3_key}"

In [34]:
endpoint_config_name = "endpoint-x-beagle"
endpoint_name = "x-beagle-endpoint"
inference_component_name = "inference-component-x-beagle"
variant_name = "variant-x-beagle"

sagemaker_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ExecutionRoleArn = "arn:aws:iam::907729080149:role/SagemakerFullAccess",
    ProductionVariants = [
        {
            "VariantName": variant_name,
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": 1,
                "MaxInstanceCount": 2,
            },
        }
    ],
)

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:ap-northeast-2:907729080149:endpoint-config/endpoint-x-beagle".

In [35]:
sagemaker_client.create_endpoint(
    EndpointName = endpoint_name,
    EndpointConfigName = endpoint_config_name,
)

ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:ap-northeast-2:907729080149:endpoint/x-beagle-endpoint".

In [36]:
import sagemaker
predictor_reuse=sagemaker.predictor.Predictor(
    endpoint_name="x-beagle-endpoint",
    sagemaker_session=sagemaker.Session(),
    serializer=sagemaker.serializers.CSVSerializer()
)

In [39]:
endpoint_config_name = "endpoint-x-beagle"
endpoint_name = "x-beagle-endpoint"
inference_component_name = "inference-component-x-beagle"
variant_name = "variant-x-beagle"

sagemaker_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ExecutionRoleArn = "arn:aws:iam::907729080149:role/SagemakerFullAccess",
    ProductionVariants = [
        {
            "VariantName": variant_name,
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": 1,
                "MaxInstanceCount": 2,
            },
        }
    ],
)

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:ap-northeast-2:907729080149:endpoint-config/endpoint-x-beagle".

In [40]:
sagemaker_client.update_endpoint(
    EndpointName="x-beagle-endpoint",
    EndpointConfigName='endpoint-x-beagle2'
)

{'EndpointArn': 'arn:aws:sagemaker:ap-northeast-2:907729080149:endpoint/x-beagle-endpoint',
 'ResponseMetadata': {'RequestId': 'fbc8e020-112a-412a-ad38-14edf1c00718',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fbc8e020-112a-412a-ad38-14edf1c00718',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Fri, 24 May 2024 05:57:11 GMT'},
  'RetryAttempts': 0}}

In [41]:
sagemaker_client.create_inference_component(
    InferenceComponentName = inference_component_name,
    EndpointName = endpoint_name,
    VariantName = variant_name,
    Specification = {
        "Container": {
            "Image": "763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference:2.0.1-cpu-py310",
            "ArtifactUrl": model_url,
        },
        "ComputeResourceRequirements": {
            "NumberOfCpuCoresRequired": 1, 
            "MinMemoryRequiredInMb": 1024
        }
    },
    RuntimeConfig = {"CopyCount": 2}
)

ClientError: An error occurred (ValidationException) when calling the CreateInferenceComponent operation: Cannot start inference component deployment on current endpoint status UPDATING. Please ensure endpoint is InService status to update or delete inference component, and InService or Creating status to create inference component.

In [43]:
pip install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.219.0
    Uninstalling sagemaker-2.219.0:
      Successfully uninstalled sagemaker-2.219.0
Note: you may need to restart the kernel to use updated packages.


In [45]:
from sagemaker import image_uris
image_uris.retrieve(framework='pytorch',region='ap-northeast-2',version='2.1.0',py_version='py310',image_scope='inference', instance_type='ml.g4dn.4xlarge')

'763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference:2.1.0-gpu-py310'